In [1]:
# Importo las librerias
import pandas as pd
import spacy
from tqdm.notebook import tqdm
import re
# si no instalo esto no puedo seguir despues
!pip install spacy &> /dev/null
!python -m spacy download es_core_news_lg &> /dev/null
from collections import Counter
import nltk
import seaborn as sns

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path='/content/drive/MyDrive/xxxxxxxxxxxxxxxxxxxxxxxxxx/'
df_sentencias = pd.read_json(path + 'sentences_XXXXXXXXXXXXXXXXXXX.jsonl',lines=True)
df_sentencias['bias'] = [1 if len(x) > 0 else 0 for x in df_sentencias['highlight']]
aa = df_sentencias.groupby('doc')[['bias']].sum()
df = df_sentencias[df_sentencias['doc'].isin(aa[aa['bias'] > 0].index)]
df

In [ ]:
# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?¿\]\%', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

#data_clean = pd.DataFrame(df.text.apply(round1))
df['text_clean'] = df.text.apply(round1)

# Apply a second round of cleaning
def clean_text_round2(text):
    
    text = re.sub('[‘’“”…«»°]', '', text)
    text = re.sub('\n', ' ', text)
    text = " ".join(text.split())
    return text

round2 = lambda x: clean_text_round2(x)

#data_clean = pd.DataFrame(data_clean.text.apply(round2))
#data_clean
df['text_clean'] = df.text_clean.apply(round2)


In [ ]:
df['text_clean'] = df['text_clean'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3]))
df

In [ ]:
nltk.download('stopwords')
stopword = nltk.corpus.stopwords.words('spanish')
df['text_clean'] = df['text_clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopword)]))



In [ ]:

from collections import Counter
a = Counter(" ".join(df["text_clean"]).split()).most_common(50)
for v in a:
  stopword.append((v[0])) ## 30


In [ ]:
df['text_clean'] = df['text_clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopword)]))
df

In [11]:
from nltk import SnowballStemmer #importo el modulo de stemmer

nlp = spacy.load("es_core_news_lg")
ss = SnowballStemmer(u'spanish')

In [ ]:
# no necesariamente el procesamiento que está acá es definitivo, era solo porque como recorría todo una vez, mejor que se
# haga todo junto

from collections import deque
import os

df_proc = pd.DataFrame(columns=['doc','text_original','text_clean','verbs','adj','nouns','nouns_masc','nouns_fem','masc','fem','verbs_lemmas','adj_lemmas','adj_lemmas_gender',
                                'adj_stems', 'text_stem','text_lemma','text_lemma_gender','text_stem_only_adj_verb','label'])
df_proc

for i in tqdm(range(len(df_proc),len(df))):
    pdf = df['doc'].values[i]
    parrafo = df['text_clean'].values[i].strip().lower()
    parrafo = re.sub('\.+','',parrafo)
    doc = nlp(parrafo)
    v = deque() # crear el dict acá capaz que sea más limpio para el resto del código...
    v_lemma = deque()
    a = deque()
    a_lemma = deque()
    a_lemma_g = deque()
    a_stem = deque()
    lemmas = deque()
    lemmas_g = deque()
    stems = deque()
    nouns = deque()
    stem_g_only = deque()
    masc = deque()
    fem = deque()
    nouns_masc = deque()
    nouns_fem = deque()
    for s in doc.sents:
        for t in s:
            if 'PUNCT' == t.pos_: #ver si está bien
                continue
            gen = t.morph.to_dict().get('Gender')
            lemmas.append(t.lemma_)
            lemmas_g.append(t.lemma_)
            stems.append(ss.stem(t.text))
            stem_g_only.append(t.text)
            if t.tag_.startswith('V'):
                v.append(t.text)
                v_lemma.append(t.lemma_)
                stem_g_only[-1] = t.lemma_
            elif t.tag_.startswith('ADJ'):
                    a.append(t.text)
                    a_stem.append(stems[-1])
                    a_lemma.append(t.lemma_)
                    a_lemma_g.append(t.lemma_)
                    stem_g_only[-1] = stems[-1]
                    if t.lemma_.endswith('o') or t.lemma_.endswith('a'): # tratar de sacar el género
                        a_lemma_g[-1] = t.lemma_[0:-1]
                        lemmas_g[-1] = t.lemma_[0:-1]
            elif t.tag_.startswith('NOUN'):
                nouns.append(t.text)
                if gen is not None:
                    if gen == 'Masc':
                        nouns_masc.append(t.text)
                    else:
                        nouns_fem.append(t.text)

            if gen is not None:
                if gen == 'Masc':
                    masc.append(t.text)
                else:
                    fem.append(t.text)

    df_proc = pd.concat([df_proc,
                          pd.DataFrame([{'doc': pdf, 'text_original' : df['text'].values[i].strip(), 'text_clean':parrafo, 'verbs':list(v),'adj':list(a),'nouns':list(nouns),'nouns_masc':list(nouns_masc),'nouns_fem':list(nouns_fem),'masc':list(masc),'fem':list(fem),'verbs_lemmas':list(v_lemma),'adj_stems':list(a_stem),'adj_lemmas':list(a_lemma),'adj_lemmas_gender':list(a_lemma_g),'text_stem':list(stems),'text_lemma':list(lemmas),'text_lemma_gender':list(lemmas_g), 'text_stem_gender_only_adj_verb':list(stem_g_only),'label':df['bias'].values[i]}])
                         ],axis=0)

# TODO: Se puede guardar en un pickle para no tener que procesar todo de nuevo, lo que puede ser largo

In [ ]:
df_proc

In [ ]:
#saving the df into pickle format
df_proc.to_pickle('df_proc_cr_sentence.pkl')
dftest = pd.read_pickle('df_proc_cr_sentence.pkl')
dftest.info()
dftest.info()

In [ ]:
def merge_label_sesgo(row):
   # print (row[2])
    if len(row[2]) > 0: #si hay algo en el campo hig considero q tiene sesgo
       # print('fdf')
        label = 1
    else:
        label = 0
    return label